# Getting to know RECON2 with FBA and FVA

Authors: Thierry D.G.A. Mondeel & Hans V. Westerhoff  <br/>
University of Amsterdam <br/>2016

Loading the model may take 10 seconds. 

In [ ]:
import cameo # the constraint-based modeling package
from cobrapyTools import * # a collection of useful scripts by Thierry
import pandas as pd # for nice tables
pd.set_option('display.max_colwidth', -1)

%time M = cameo.load_model("models/Recon2.v04_pythonComp.json",sanitize=False)
model = M.copy() # this way we can edit model but leave M unaltered
model

## Model properties

In [ ]:
model.medium

The reactions in RECON2 are categorized in various compartments. 

In [ ]:
model.compartments

### Question
Do you know the biological role of all these compartments?

See here for more information: http://www.ncbi.nlm.nih.gov/books/NBK26907/

### Assignment
Write a for loop that prints the number of metabolites in each compartment.

**Tip:** Within the for loop make use of a list comprehension that finds metabolites in a specific compartment.

## Enzyme encoding genes

Find the gene encoding the enzyme for a particular reaction (if it is known). We will take the reaction converting phenylalanine to tyrosine.

In [ ]:
model.reactions.PHETHPTOX2

In [ ]:
print model.reactions.PHETHPTOX2.gene_name_reaction_rule

### Assignment
Look this gene ID 5053 up in the uniprot database and check that it is correctly annotated. URL: http://www.uniprot.org

Also check that the reaction itself is correct.

## Flux bounds and printing reaction properties

In [ ]:
model.reactions.r0399

### Assignment
Compare reaction r0399 and the earlier PHETHPTOX2 reaction. What is their relationship? Are they encoded by the same gene or different genes? Check this.

### Assignment
Try to print in the cell below all the reactions (not just their names but their reaction equations) that phenylalanine engages in

## Performing Flux Balance Analysis

In [ ]:
model.objective.expression

In [ ]:
sol = cameo.pfba(model)
print 'Solution status: {}'.format(model.solution.status) # see if everything went OK
print 'Objective value: {}'.format(model.solution.fluxes['biomass_reaction'])

### Assignment
Write a list comprehension in the cell below that calculates the total flux, i.e. the sum of all fluxes. We already added the print command for you.

You can check that you did it right by asking for the value of sol.objective_value

In [ ]:
total_flux = sum([]) # write a list comprehension here, make use of sol.data_frame.flux
# make sure you take the absolute value of the flux (use abs()) so that negative fluxes (going in the backward direction) 
# are not reducing the total flux value
print 'Total flux: {}'.format(total_flux)

### Are essential amino acids essential?

We will now look into which reactions the model predicts to be essential for growth. 

### Assignment method 1: look at the output of pFBA.
Use the cell below to print all non-zero flux values. Try to look at the exchange reactions (starting with "EX_") in particular. They represent what medium components were used. Are all 9 essential amino acids actually predicted to be needed by the model?

In [ ]:
fbaTable = pd.DataFrame( { 'Name' : list( model.reactions ),
                'Lb' : [model.reactions[i].lower_bound for i in range(len(model.reactions)) ],
                'Ub' : [model.reactions[i].upper_bound for i in range(len(model.reactions)) ],
                'Flux' : model.solution.x  } )
fbaTable = fbaTable[['Name','Flux','Lb','Ub']]
fbaTable[abs(fbaTable.Flux) > 0]

### Assignment Method 2: Calculating the essential genes using flux variability analysis
This is built-in in cameo under the model.essential_reactions() function. See the code cell below.

Are the "essential" amino acids predicted to be essential?

Compare the number of essential reactions predicted here, with the number of reactions you got when doing pFBA? 
Why did pFBA give you more reactions?

In [ ]:
model = M.copy()
ess_rxns = model.essential_reactions()

print '{} reactions are essential for growth\n\n'.format(len(ess_rxns))

fbaTable = pd.DataFrame( { 'ID' : [rxn.id for rxn in ess_rxns],
                'Name' : [rxn.name for rxn in ess_rxns],
                'Reaction' : [rxn.reaction for rxn in ess_rxns]  } )
fbaTable[['ID','Name','Reaction']]

### Question: Are the PHETHPTOX2 and r0399 reactions essential? 
If yes, why? If not, why not?

### Deletions
We will now block r0399 and see what happens. 

In [ ]:
model.reactions.r0399.upper_bound = 0; model.reactions.r0399.lower_bound = 0;

ess_rxns2 = model.essential_reactions()

print '{} reactions are essential for growth\n\n'.format(len(ess_rxns2))

fbaTable = pd.DataFrame( { 'ID' : [rxn.id for rxn in ess_rxns2],
                'Name' : [rxn.name for rxn in ess_rxns2],
                'Reaction' : [rxn.reaction for rxn in ess_rxns2]  } )
fbaTable[['ID','Name','Reaction']]

### Question: What changed and why?

These reactions are newly essential

In [ ]:
for rxn in ess_rxns2:
    if rxn not in ess_rxns :
        print '{} \t {} \t {}'.format(rxn.id,rxn.name,rxn.reaction)

### Assignment
Knockout the gene encoding both the PHETHPTOX2 and r0399 reactions and see what happens if you optimize for growth. What do you expect? Is that what you see?

We already wrote the code that knocks out the gene for you. You should add the FBA simulation and print the flux through the biomass reaction

In [ ]:
model = M.copy() # start fresh

print model.reactions.PHETHPTOX2.genes # list the gene

model.genes.g1573.knock_out() # knock it out

# check that it worked
model.reactions.PHETHPTOX2.upper_bound 